## Steps to Complete Project Part 1:

### Step 1: Set up a Google Sheet and obtain credentials for Google Sheet API

    Get the Service Account Email - gsheet@project-XXXX.iam.gserviceaccount.com and Google Sheet API Key also.
    
### Step 2: Install and Import All the Libaraies Use for this Project

In [1]:
pip install pandas yfinance gspread

Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd
import yfinance as yf
import gspread   

In [2]:
import time

In [3]:
gc = gspread.service_account(filename= "project-383413-c1667ff4d679.json")

### Step 3: Access the Gsheet and worksheets

In [4]:
# Open the Google Sheet
spreadsheet1 = gc.open("Automate Stock Selection For SST method (Part 1)")
Todays_Final_List = spreadsheet1.worksheet("Today's Final List") 
Nifty100_list = spreadsheet1.worksheet("Nifty100")
Selected_list = spreadsheet1.worksheet("Selected List")

Nifty100_list

<Worksheet 'Nifty100' id:0>

### Step 4: Write Python code to fetch Nifty 100 stock data from YFinance and Update in the Sheet

In [6]:
start_row_Nifty100 = 2
end_row_Nifty100 = 101
column3_Nifty100 = 3

cell_values_col3 = Nifty100_list.col_values(column3_Nifty100)[start_row_Nifty100 - 1:end_row_Nifty100]

column4_Nifty100 = 4
column5_Nifty100 = 5
column6_Nifty100 = 6
column7_Nifty100 = 7

# Loop through the stock symbols and fetch data from Yahoo Finance
for i in range(len(cell_values_col3)):
    value1 = cell_values_col3[i]
    
    # Define the stock symbol
    symbol = value1
    #print(f"Fetching data for stock symbol {symbol}...")
    
    try:
        stock_data = yf.Ticker(symbol)
        
        # Get the current market price
        current_price = stock_data.info.get("regularMarketPrice", None)
        if current_price is None:
            current_price = stock_data.info.get("regularMarketOpen", None) # Use regularMarketOpen as fallback
            if current_price is None:
                current_price = stock_data.info.get("regularMarketPreviousClose", None) # Use regularMarketPreviousClose as fallback

        # Get the 20 days low and 20 days high
        history_data = stock_data.history(period="20d")
        low_20d = history_data["Low"].min()
        high_20d = history_data["High"].max()

        # Get the latest day's low
        latest_low = history_data.iloc[-1]["Low"]

        # Fetch the latest day's close price
        stock_data = yf.download(symbol, period="1d", interval="1d", progress=False)
        latest_close_price = stock_data.iloc[-1]["Close"]
        
        #print(f"Stock Symbol: {symbol}")
        #print(f"Current Market Price: {current_price}")
        #print(f"20 Days Low: {low_20d}")
        #print(f"20 Days High: {high_20d}")
        #print(f"Latest Day's Low: {latest_low}")
        #print(f"Latest Day's Close Price: {latest_close_price}")
        #print("-------------------------------")
              
        row = start_row_Nifty100 + i
        col1 = column4_Nifty100
        col2 = column5_Nifty100
        col3 = column6_Nifty100
        col4 = column7_Nifty100
        #print("row value:", row)
        #print("column value:", col1)
        
        # Wait for 5 seconds before fetching data for the next stock
        time.sleep(5) 
        
        Nifty100_list.update_cell(row, col1, latest_close_price)
        Nifty100_list.update_cell(row, col2, high_20d)
        Nifty100_list.update_cell(row, col3, low_20d)
        Nifty100_list.update_cell(row, col4, latest_low)
        #print("Updated row:", row)
        #print("Updated column 1 value:", latest_close_price)
        #print("Updated column 2 value:", high_20d)
        #print("Updated column 3 value:", low_20d)
        #print("Updated column 4 value:", latest_low)
         
    except:
        print(f"Unable to fetch data for stock symbol {symbol}")

### Step 5: Write formulas in the Google Sheet to analyze the data and select stocks based on CMP, 20-day high, and today's low.

###### Excel Sheet If Formula : Syntax :- =IF(logical_test, [value_if_true], [value_if_false]) 
###### = IF((F3(20 Days  low)>=G3(Today Low)),"Start GTT order at 20 days high and update regularly","Do not start GTT wait for best time")

Rule :- If any Stocks (20 Days low price) >= (Today Low Price)
Using the Rule, Stock get short listed from Nifty100 worksheet and The same stock are display in the Today's Final List workseet by using the 

###### =iferror(filter(Nifty100!A:A,Nifty100!H:H="Start GTT order at 20 days high and update regularly"),"")


### Step 6: Create a separate worksheet to display the selected stocks

In [7]:
Selected_list

<Worksheet 'Selected List' id:413928241>

In [12]:
# Fetch values from column 5 in rows 5 to 54
start_row_sheet1 = 5
end_row_sheet1 = 54
column1_sheet1 = 2
column2_sheet1 = 3
cell_values_col1 = Todays_Final_List.col_values(column1_sheet1)[start_row_sheet1 - 1:end_row_sheet1]
cell_values_col2 = Todays_Final_List.col_values(column2_sheet1)[start_row_sheet1 - 1:end_row_sheet1]
print("Cell value:", cell_values_col1)

start_row_sheet3 = 3
end_row_sheet3 = 54
column1_sheet3 = 2

for i in range(len(cell_values_col1)):
    if not cell_values_col2[i]:  # Check if the value in 'Today's Final List' worksheet is empty
        break  # Break the loop when an empty cell is encountered in 'Today's Final List' worksheet
    value2 = cell_values_col2[i]
    value1 = cell_values_col1[i]
    for i in range(50):  # Loop 50 times
        row = start_row_sheet3 + i
        col2 = column1_sheet3
        col1 = column1_sheet3 -1
        print("row value:", row)
        print("column value:", col1)
        print("value:", value1)
        cell1 = Selected_list.cell(row, col2).value
        if cell1 == value2:
            print("Value already present in GTT tracking. Breaking loop.")
            break
        elif not cell1:
            Selected_list.update_cell(row, col1, value1)
            Selected_list.update_cell(row, col2, value2)
            print("Updated row:", row)
            print("Updated column 1 value:", value1)
            break

Cell value: ['Godrej Consumer Products Ltd.', 'Hindustan Unilever Ltd.', 'Tech Mahindra Ltd.']
row value: 3
column value: 1
value: Godrej Consumer Products Ltd.
row value: 4
column value: 1
value: Godrej Consumer Products Ltd.
row value: 5
column value: 1
value: Godrej Consumer Products Ltd.
row value: 6
column value: 1
value: Godrej Consumer Products Ltd.
row value: 7
column value: 1
value: Godrej Consumer Products Ltd.
row value: 8
column value: 1
value: Godrej Consumer Products Ltd.
row value: 9
column value: 1
value: Godrej Consumer Products Ltd.
row value: 10
column value: 1
value: Godrej Consumer Products Ltd.
Updated row: 10
Updated column 1 value: Godrej Consumer Products Ltd.
row value: 3
column value: 1
value: Hindustan Unilever Ltd.
row value: 4
column value: 1
value: Hindustan Unilever Ltd.
row value: 5
column value: 1
value: Hindustan Unilever Ltd.
row value: 6
column value: 1
value: Hindustan Unilever Ltd.
row value: 7
column value: 1
value: Hindustan Unilever Ltd.
row va

In [11]:
# Fetch values from column 5 in rows 5 to 54
start_row_sheet1 = 5
end_row_sheet1 = 54
column1_sheet1 = 2
column2_sheet1 = 3
cell_values_col1 = Todays_Final_List.col_values(column1_sheet1)[start_row_sheet1 - 1:end_row_sheet1]
cell_values_col2 = Todays_Final_List.col_values(column2_sheet1)[start_row_sheet1 - 1:end_row_sheet1]
print("Cell value:", cell_values_col1)

start_row_sheet3 = 3
end_row_sheet3 = 54
column1_sheet3 = 2

for i in range(len(cell_values_col1)):
    if not cell_values_col2[i]:  # Check if the value in 'Today's Final List' worksheet is empty
        break  # Break the loop when an empty cell is encountered in 'Today's Final List' worksheet
    value1 = cell_values_col1[i]
    value2 = cell_values_col2[i]
    
    is_present = False  # Initialize flag variable
    for i in range(50):  # Loop 50 times
        row = start_row_sheet3 + i
        col1 = column1_sheet3 - 1
        col2 = column1_sheet3
        
        print("row value:", row)
        print("column value:", col1)
        print("value:", value1)
        cell1 = Selected_list.cell(row, col1).value
        print("cell1", cell1)
        print("value2", value2)
        if cell2 == value2:
            print("Value already present in GTT tracking. Breaking loop.")
            is_present = True  # Update flag variable
            break
        elif not cell1:
            Selected_list.update_cell(row, col1, value1)
            Selected_list.update_cell(row, col2, value2)
            print("Updated row:", row)
            print("Updated column 1 value:", value1)
            is_present = True  # Update flag variable
            break
    if is_present:  # Check flag variable
        continue  # Skip updating the stock data in Sheet 3 if the value is already present

Cell value: ['Godrej Consumer Products Ltd.', 'Hindustan Unilever Ltd.', 'Tech Mahindra Ltd.']
row value: 3
column value: 1
value: Godrej Consumer Products Ltd.
cell1 ABB India Ltd.
value2 GODREJCP.NS
row value: 4
column value: 1
value: Godrej Consumer Products Ltd.
cell1 FSN E-Commerce Ventures Ltd.
value2 GODREJCP.NS
row value: 5
column value: 1
value: Godrej Consumer Products Ltd.
cell1 ICICI Lombard General Insurance Company Ltd.
value2 GODREJCP.NS
row value: 6
column value: 1
value: Godrej Consumer Products Ltd.
cell1 LTIMindtree Ltd.
value2 GODREJCP.NS
row value: 7
column value: 1
value: Godrej Consumer Products Ltd.
cell1 Shree Cement Ltd.
value2 GODREJCP.NS
row value: 8
column value: 1
value: Godrej Consumer Products Ltd.
cell1 Siemens Ltd.
value2 GODREJCP.NS
row value: 9
column value: 1
value: Godrej Consumer Products Ltd.
cell1 Tech Mahindra Ltd.
value2 GODREJCP.NS
row value: 10
column value: 1
value: Godrej Consumer Products Ltd.
cell1 Godrej Consumer Products Ltd.
value2 GO

#### Step 7: Set up a cron job to run the script every day at 9 pm to update the selected stocks list.
this code in Part 2
